In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import socket
import time

In [ ]:
blank_img = np.zeros(shape=(512,512,3))
blank_img.shape

In [ ]:
# variables
lmb_pressed = False

In [ ]:
# configure tcp client
s = socket.socket()
port = 8080
s.connect(('127.0.0.1', port))

In [ ]:
# clreate image with rectangles
def draw_rectangles(img_arg):
    
    bgr_img = np.copy(img_arg)
    
    x_max = int( bgr_img.shape[1] )
    y_max = int( bgr_img.shape[0] )
    x_ctr = int(x_max/2)
    y_ctr = int(y_max/2)
    
    # draw circle in the center
    cv2.circle(bgr_img, center=(x_ctr,y_ctr), radius=10, color=(0, 255, 0), thickness=-1)
    
    # draw red lines
    cv2.line(bgr_img, pt1=(x_ctr, 0), pt2=(x_ctr, y_max), color=(0,0,255), thickness=1)
    cv2.line(bgr_img, pt1=(0, y_ctr), pt2=(x_max, y_ctr), color=(0,0,255), thickness=1)
    
    # draw rectangles
    cv2.rectangle(bgr_img,
                 pt1=( int((4/5)*x_ctr), int((4/5)*y_ctr)),
                 pt2=( int((6/5)*x_ctr), int((6/5)*y_ctr)),
                 color=(150,150,150),
                 thickness=2)
    
    cv2.rectangle(bgr_img,
                 pt1=( int((3/5)*x_ctr), int((3/5)*y_ctr)),
                 pt2=( int((7/5)*x_ctr), int((7/5)*y_ctr)),
                 color=(150,150,150),
                 thickness=2)
    
    cv2.rectangle(bgr_img,
                 pt1=( int((2/5)*x_ctr), int((2/5)*y_ctr)),
                 pt2=( int((8/5)*x_ctr), int((8/5)*y_ctr)),
                 color=(150,150,150),
                 thickness=2)
    
    cv2.rectangle(bgr_img,
                 pt1=( int((1/5)*x_ctr), int((1/5)*y_ctr)),
                 pt2=( int((9/5)*x_ctr), int((9/5)*y_ctr)),
                 color=(150,0,0),
                 thickness=2)
    
    
    return bgr_img

In [ ]:
# calculation of timer ticks for STM32
# temporary solution to avoid changes in FreeRTOS
def send_st_temp_solution(steer, throttle):
    
    s_min = 1960
    s_max = 2730
    
    t_min = 1895
    t_max = 3120
    
    # from y = ax + b 
    s_result = int( ((s_max-s_min)/100)*steer + s_min)
    t_result = int( ((t_min-t_max)/100)*throttle + t_max)
    
    msg = 's' + str(s_result) + 't' + str(t_result)
    
    s.send(bytes(str(msg), 'utf8'))
    
    print('\r',msg, end='')
    

In [ ]:
# calculate x and y from mouse position
def get_st(x, y, img_shape):
    
    x_ctr = int(img_shape[1]/2)
    y_ctr = int(img_shape[0]/2)
    max_value = 100
    
    # min = 0 at 1/5 of the distance btw. ctr and frame
    x_min = int((1/5)*x_ctr)
    y_min = int((1/5)*y_ctr)
    
    # max = 100 at 9/5 of the distance btw. ctr and frame
    x_max = int((9/5)*x_ctr)
    y_max = int((9/5)*y_ctr)
    
    # controller resolution: 0 to max_value
    res = max_value/2
    steer = int(res*(x - x_min)/(x_ctr - x_min))
    throttle = int(res*(y - y_max)/(y_ctr - y_max))
    
    if throttle >= res*2:
        throttle  = int(res*2 - 1)
    elif throttle  < 0:
        throttle  = 0
    
    if steer >= res*2:
        steer = int(res*2 - 1)
    elif steer < 0:
        steer = 0
    
    send_st_temp_solution(steer, throttle)
    
    

In [ ]:
# draw circle using mouse events
def draw_circle(event, x, y, flags, param):
    
    time.sleep(0.03)
    
    global lmb_pressed, img
    x_ctr = int(img.shape[1]/2)
    y_ctr = int(img.shape[0]/2)
    
    if event == cv2.EVENT_LBUTTONDOWN:
        lmb_pressed = True
        img = rect_img.copy()
        cv2.circle(img, center=(x_ctr,y_ctr), radius=10, color=(0, 0, 0), thickness=-1)
        cv2.circle(img, center=(x,y), radius=10, color=(0, 255, 0), thickness=-1)
        #print('\r',x, y, sep=' ', end='')
        get_st(x, y, img.shape)
    
    elif event == cv2.EVENT_LBUTTONUP:
        lmb_pressed = False
        img = rect_img.copy()
        cv2.circle(img, center=(x_ctr,y_ctr), radius=10, color=(0, 255, 0), thickness=-1)
        get_st(x_ctr, y_ctr, img.shape)
        get_st(x_ctr, y_ctr, img.shape)
        get_st(x_ctr, y_ctr, img.shape)
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if lmb_pressed == True:
            img = rect_img.copy()
            cv2.circle(img, center=(x_ctr,y_ctr), radius=10, color=(0, 0, 0), thickness=-1)
            cv2.circle(img, center=(x,y), radius=10, color=(0,255,0), thickness=-1)
            #print('\r',x, y, sep=' ', end='')
            get_st(x, y, img.shape)
                    


In [ ]:
rect_img = draw_rectangles(blank_img)

# connect to call back
cv2.namedWindow('RC_control')
cv2.setMouseCallback('RC_control', draw_circle)

img = rect_img.copy()

while True:
    cv2.imshow('RC_control', img)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()